In [ ]:
import pandas as pd
from wordcloud import WordCloud
from IPython.display import Image
from PIL import Image as imgWordcloud
import numpy as np
import re

pd.options.display.max_colwidth = 1000

Load the data

In [ ]:
data = pd.read_csv('../MyData/labeled_data.csv')
size = len(data)
data[:10]

Let's have a peek at our classes

In [ ]:
print(data["class"].unique())

Our classes are labeled with numbers, for our ease we will map each number to a word. We have 3 classes, hate speech, offensive langaue and nuetral speech.

In [ ]:
data = data.drop(labels=['count', 'hate_speech', 'offensive_language', 'neither'], axis='columns')

# Change classes from numbers to words. Easier for me
data['class'] = data['class'].map({0:'hate speech', 
                                   1:'offensive language', 
                                   2:'neither'})
print(data['class'].unique())

We will now perform some simple preprocessing by removing info regarding retweets, that appear in the beggining of eacg tweet since this is connsidered noise. 

In [ ]:
def preprocessText(initText):
    processedText = initText
    
    # Tweets are loaded between quotation marks and we will remove them
    #Remove the " from the begining
    processedText = re.sub(r'^\"+', '', processedText)
    
    # And the end
    processedText = re.sub(r'\"+$', '', processedText)
    
    # Remove all the random spaces at the begining 
    processedText = re.sub(r'^ +', '', processedText)

    # And any trailing spaces too 
    processedText = re.sub(r' +$', '', processedText)
    
    return processedText

def clear_first(text):
    processedText = text
    # Remove the retweet info
    processedText = re.sub(r'^([!| |\"|RT]*(\B@\w\w+)* *\w*)*:', '',
                           processedText)
    return processedText

In [ ]:
for i, row in data.iterrows():
    initText = preprocessText(row["tweet"])
    data.iat[i, 2] = initText

data[:10]

Convert HTML

In [ ]:
from html.parser import HTMLParser
import lxml, lxml.html

for i, row in data.iterrows():
    text = lxml.html.document_fromstring(row["tweet"]).text_content()
    data.iat[i, 2] = text

data[:10]

Remove the rewteet info 

In [ ]:
for i, row in data.iterrows():
    text = clear_first(row["tweet"])
    data.iat[i, 2] = text

data[:10]

We will use two classes for our project, hate speech and neutral so we will merge hate speech and offensive langauge classes to one hate speech class


In [ ]:
data['class'] = data['class'].map({'hate speech': 'hate speech', 
                                   'offensive language': 'hate speech', 
                                   'neutral': 'neutral'})

data[:10]

Save all the data to csv

In [ ]:
data.to_csv("twitter_hate_data.csv")